# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [ ]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.1/378.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.2 MB/s eta 0:00:00


We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [ ]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [3]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-03-04 13:37:27--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0.01s   

2025-03-04 13:37:28 (1.33 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2025-03-04 13:37:28--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [5]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 3, 1, 13, 37, 43, 508795)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [6]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [7]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [9]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [10]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [11]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, based on the reviews provided, it seems that people generally liked John Wick.'

In [12]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is one review with a rating of 10. Here is the URL to that review:\n- Review: A Masterpiece & Brilliant Sequel\n  URL: [Read the Expert Review](/review/rw4854296/?ref_=tt_urv)'

In [13]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the story follows an ex-hitman who comes out of retirement to seek revenge against the gangsters who killed his dog and took everything from him. The plot revolves around his quest for vengeance and the violent consequences that ensue.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [14]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [15]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [16]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Opinions on John Wick seem to vary widely. Some people really enjoyed the action and style of the movie, while others found it lacking in substance and plot.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"I'm sorry but none of the reviews have a rating of 10."

In [18]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character, played by Keanu Reeves, goes on a violent and action-packed quest for revenge. He is a former hitman seeking retribution for the killing of his beloved dog, a final gift from his deceased wife. The movie is known for its intense action scenes and stylish choreography.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [19]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [20]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." The URL to that review is: /review/rw4854296/?ref_=tt_urv'

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, after resolving issues with the Russian mafia, John Wick is forced back into action when mobster Santino D'Antonio asks him to kill his sister in Rome. When John completes the task, a contract is put on his life, leading to a series of intense and action-packed events."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [24]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [25]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [26]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, people generally liked John Wick. The film was praised for its slickness, brilliant action sequences, entertaining story, impressive choreography, and lead performance by Keanu Reeves. Overall, it was well-received as a high-quality action film.'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\''

In [28]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick: Chapter 2, former hitman John Wick is pulled back into the world of assassins when an Italian crime lord forces him to fulfill a debt by carrying out a hit. This sets off a chain of events that leads to a contract being placed on John Wick's life, resulting in intense action and high-stakes confrontations."

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [29]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [30]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/br/w3rfh7rx301551532ps5zfgm0000gn/T/ipykernel_87393/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [31]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [32]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [33]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [34]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"People's opinions on John Wick seem to be divided. Some people really enjoy the series and consider it consistent and well-received, while others have strong negative opinions about it, calling it horrible."

In [35]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10. Here is the URL to that review: '/review/rw4854296/?ref_=tt_urv'"

In [36]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the protagonist, played by Keanu Reeves, is a retired assassin who comes out of retirement after someone kills his dog. He seeks revenge by going on a killing spree after his car is stolen. Later in the sequel, he is forced back into the world of assassins when an Italian criminal calls in a favor. John Wick travels to Italy, Canada, and Manhattan, killing numerous assassins along the way in a quest for vengeance.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [37]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [38]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [39]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." Here is the URL to that review: /review/rw4854296/?ref_=tt_urv'

In [41]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, an ex-hitman comes out of retirement to seek vengeance on the gangsters that killed his dog and took everything from him. This sets off a series of violent events as he faces off against various assassins and criminals who are after him due to a high bounty on his head. The movie is known for its intense action, stylish stunts, and gripping storyline of revenge.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [ ]:
#!pip install -qU langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [42]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [43]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [44]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [45]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [46]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [47]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the reviews provided.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\''

In [49]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character seeks revenge on the people who killed his dog and took everything from him. This simple premise leads to intense action sequences and stylish stunts throughout the movie.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [ ]:
# Import necessary libraries
from ragas.testset import TestsetGenerator
from langchain_openai import ChatOpenAI
import pandas as pd
from tqdm import tqdm

In [54]:

# Set up the LLM for test generation
llm = ChatOpenAI(model="gpt-4o-mini")

# Create a test set generator
testset_generator = TestsetGenerator.from_langchain(llm, embedding_model=embeddings)

# Generate synthetic test questions based on our documents
print("Generating synthetic test questions...")
test_dataset = testset_generator.generate_with_langchain_docs(
    documents=documents,
    testset_size=10
)

test_dataset.to_pandas()


Generating synthetic test questions...


Applying SummaryExtractor:   0%|          | 0/44 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/100 [00:00<?, ?it/s]

Node 6bc5b11f-13a6-4b84-a2bb-70e40d4f3836 does not have a summary. Skipping filtering.
Node 5bc45339-c207-4e57-b776-2375477a6094 does not have a summary. Skipping filtering.
Node d9f9bb30-a1dc-4e2a-8220-fd5e9abbd1f4 does not have a summary. Skipping filtering.
Node 740aacc0-bf3a-405f-abcf-4f14832868dd does not have a summary. Skipping filtering.
Node 365b872a-7a81-4bbe-b09a-334fa98976d4 does not have a summary. Skipping filtering.
Node 4d72cb77-5996-491b-b768-8064b63f27f8 does not have a summary. Skipping filtering.
Node 080d5d55-5fc4-422b-8beb-4f8a96c5f68a does not have a summary. Skipping filtering.
Node bc23f6ef-16b0-4e68-a177-3e94bcc58ac4 does not have a summary. Skipping filtering.
Node 2590e7dd-e8d0-4a97-b713-3a2b60360ca1 does not have a summary. Skipping filtering.
Node fb2bd858-a21b-40fa-9755-13a5f36ab531 does not have a summary. Skipping filtering.
Node 5e301302-5949-4f6d-bc54-13ff21cbc826 does not have a summary. Skipping filtering.
Node c675a80a-c8e0-4488-8140-e9fb428b058d d

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/244 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How does Keanu Reeves contribute to the appeal...,[: 0\nReview: The best way I can describe John...,Keanu Reeves contributes to the appeal of John...,single_hop_specifc_query_synthesizer
1,What are the general sentiments about the John...,[: 2\nReview: With the fourth installment scor...,"The John Wick film series, particularly the fo...",single_hop_specifc_query_synthesizer
2,What is Chad Stahelski known for in the film J...,[: 3\nReview: John wick has a very simple reve...,Chad Stahelski is known for directing John Wic...,single_hop_specifc_query_synthesizer
3,Wut makes Reeves so cool in action films?,[: 4\nReview: Though he no longer has a taste ...,"Savvy, indestructible Reeves looks right at ho...",single_hop_specifc_query_synthesizer
4,How does the theme of loss and grief influence...,[<1-hop>\n\n: 18\nReview: When the story begin...,"In the first 'John Wick' movie, the theme of l...",multi_hop_abstract_query_synthesizer
5,What are the key elements of gun shootings and...,"[<1-hop>\n\n: 0\nReview: No doubt about it, ""J...",'John Wick: Chapter 2' is celebrated for its s...,multi_hop_abstract_query_synthesizer
6,What are the contrasting opinions on the actio...,[<1-hop>\n\n: 23\nReview: Rating 10/10\nI was ...,The reviews present contrasting opinions on th...,multi_hop_abstract_query_synthesizer
7,What are the key differences in the action ele...,"[<1-hop>\n\n: 9\nReview: ""John Wick: Chapter 2...","""John Wick: Chapter 2"" is noted for being cons...",multi_hop_abstract_query_synthesizer
8,Why some people think John Wick: Chapter 3 - P...,[<1-hop>\n\n: 13\nReview: Following on from tw...,Some people think John Wick: Chapter 3 - Parab...,multi_hop_specific_query_synthesizer
9,What makes Keanu Reeves' performance in John W...,[<1-hop>\n\n: 20\nReview: John Wick is somethi...,Keanu Reeves' performance in John Wick stands ...,multi_hop_specific_query_synthesizer


In [108]:
# import our metrics
from ragas.metrics import (ContextPrecision, ContextRecall, ContextEntityRecall)
from ragas import evaluate, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
import time
from langsmith import traceable

In [110]:
@traceable
def evaluate_retriever(retrieval_chain, name, test_dataset):
    """
    Evaluate a retriever using RAGAS metrics

    Args:
        retriever: The retriever to evaluate
        name: Name of the retriever for identification
        test_dataset: The test dataset containing questions

    Returns:
        DataFrame with evaluation results
    """
    print(f"\nEvaluating {name} retriever...")

    # Process each question in the test dataset
    print(f"Retrieving contexts for {name}:")
    for test_row in test_dataset:
        response = retrieval_chain.invoke({"question" : test_row.eval_sample.user_input})
        test_row.eval_sample.response = response["response"].content
        test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
        time.sleep(2)

    # Create the evaluation dataset

    eval_dataset = EvaluationDataset.from_pandas(test_dataset.to_pandas())

    # Define the metrics to evaluate
    metrics = [
        ContextPrecision(),
        ContextRecall(),
        ContextEntityRecall()
    ]

    # Run the evaluation
    results = evaluate(
        dataset=eval_dataset,
        metrics=metrics,
        llm=LangchainLLMWrapper(llm)
    )

    # Convert results to DataFrame for easier viewing
    return results


In [100]:
# get langsmith api key
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "advanced-retrieval-evaluation"


In [101]:

os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API Key:")

In [104]:
# evaluate the naive_retriever
naive_result = evaluate_retriever(naive_retrieval_chain, "naive_retriever", test_dataset)
naive_result



Evaluating naive_retriever retriever...
Retrieving contexts for naive_retriever:


Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'context_precision': 0.8517, 'context_recall': 0.8819, 'context_entity_recall': 0.4627}

In [94]:

# eval the bm25_retrieval_chain
bm25_result = evaluate_retriever(bm25_retrieval_chain, "bm25_retriever", test_dataset)
bm25_result



Evaluating bm25_retriever retriever...
Retrieving contexts for bm25_retriever:


Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'context_precision': 0.6644, 'context_recall': 0.7542, 'context_entity_recall': 0.4384}

In [107]:
# eval the contextual_compression_retrieval_chain
contextual_compression_result = evaluate_retriever(contextual_compression_retrieval_chain, "contextual_compression_retriever", test_dataset)
contextual_compression_result


Evaluating contextual_compression_retriever retriever...
Retrieving contexts for contextual_compression_retriever:


Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'context_precision': 0.8472, 'context_recall': 0.7542, 'context_entity_recall': 0.4021}

In [111]:
# eval the parent_document_retrieval_chain
parent_document_result = evaluate_retriever(parent_document_retrieval_chain, "parent_document_retriever", test_dataset)
parent_document_result


Evaluating parent_document_retriever retriever...
Retrieving contexts for parent_document_retriever:


Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'context_precision': 0.6528, 'context_recall': 0.3833, 'context_entity_recall': 0.2968}

In [112]:
# eval the multi_query_retrieval_chain
multi_query_result = evaluate_retriever(multi_query_retrieval_chain, "multi_query_retriever", test_dataset)
multi_query_result


Evaluating multi_query_retriever retriever...
Retrieving contexts for multi_query_retriever:


Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'context_precision': 0.7904, 'context_recall': 0.8819, 'context_entity_recall': 0.4444}

In [113]:
# eval the semantic_retrieval_chain
semantic_result = evaluate_retriever(semantic_retrieval_chain, "semantic_retriever", test_dataset)
semantic_result


Evaluating semantic_retriever retriever...
Retrieving contexts for semantic_retriever:


Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'context_precision': 0.7550, 'context_recall': 0.8542, 'context_entity_recall': 0.5194}

In [114]:
# eval the ensemble_retrieval_chain
ensemble_result = evaluate_retriever(ensemble_retrieval_chain, "ensemble_retriever", test_dataset)
ensemble_result



Evaluating ensemble_retriever retriever...
Retrieving contexts for ensemble_retriever:


Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'context_precision': 0.7396, 'context_recall': 0.9583, 'context_entity_recall': 0.4428}

In [117]:
# Compile all results into a list
all_results = [
    {"retriever": "bm25_retriever", "results": bm25_result},
    {"retriever": "contextual_compression_retriever", "results": contextual_compression_result},
    {"retriever": "parent_document_retriever", "results": parent_document_result},
    {"retriever": "multi_query_retriever", "results": multi_query_result},
    {"retriever": "semantic_retriever", "results": semantic_result},
    {"retriever": "ensemble_retriever", "results": ensemble_result}
]

all_results


[{'retriever': 'bm25_retriever',
  'results': {'context_precision': 0.6644, 'context_recall': 0.7542, 'context_entity_recall': 0.4384}},
 {'retriever': 'contextual_compression_retriever',
  'results': {'context_precision': 0.8472, 'context_recall': 0.7542, 'context_entity_recall': 0.4021}},
 {'retriever': 'parent_document_retriever',
  'results': {'context_precision': 0.6528, 'context_recall': 0.3833, 'context_entity_recall': 0.2968}},
 {'retriever': 'multi_query_retriever',
  'results': {'context_precision': 0.7904, 'context_recall': 0.8819, 'context_entity_recall': 0.4444}},
 {'retriever': 'semantic_retriever',
  'results': {'context_precision': 0.7550, 'context_recall': 0.8542, 'context_entity_recall': 0.5194}},
 {'retriever': 'ensemble_retriever',
  'results': {'context_precision': 0.7396, 'context_recall': 0.9583, 'context_entity_recall': 0.4428}}]

# Analysis of Retriever Performance

Based on the evaluation results, the **ensemble_retriever** appears to be the best choice for this particular data, with the highest context recall (0.9583) among all retrievers. This means it successfully retrieves the most relevant information from the dataset.

The **multi_query_retriever** also performs well with the second-highest context recall (0.8819) and a good balance of context precision (0.7904).

The **contextual_compression_retriever** has the highest context precision (0.8472), making it the best option when accuracy of retrieved information is the priority over completeness.

The **semantic_retriever** has the highest context entity recall (0.5194), suggesting it's best at capturing specific named entities in the context.

For this John Wick movie review dataset, the ensemble_retriever would be the optimal choice as it provides the most comprehensive retrieval of relevant information, which is crucial for answering user queries about movie opinions and details.
